Все теги, которые могут встретиться, возможно найдется еще несколько, они неунифицированы во всем словаре.

In [1]:
import re
tags = '†|‡|nst|plupf|dem|prp|typ|neg|Phys|phys|Mus|m|emph|orn|impfct|a/adv|prtcl|inter|pn/prtcl|ab|ling|3p|dj|fin|naut|vi2/3|v_inv2|v_stat2|v_stat3|vi2|vt3|v_inv2|usu|vi1|vt1/2/3|vt1|Rus|pn/n|np|elec|nst|zoo|Saba|arc|pag/d|bio|med|vu|dim|cul|min|theat|Sov|mil|mus|ab|ac|Aj|Al|bk|ch|cj|clt|dg|el|Er|eu|f|fam|geo|geol|Gm|Gu|IG|Ig|ij|Im|imp|imper|Impf|iness|invar|io|I/P|ir|Isl|Ja|Ju|Ka|Ki|Kk|Kv|Le|lg|LI|lst|m/f|Me|Mg|Ml|nst|obl|on|padg|pad|instr|pag|pai|pa|pa|pf|pn|pr|prd|prg|Ps|psy|pt|px|Ra|sf|sj|sp|sst|st|tec|tex|Ti|Tu|UI|st|‡|Ka|Mti|†|©|Gd|hon|ecc|bot|Ps|num|mat|ma|chem|pp|ppa|onom|ab|v_inv|vs|Pol|np|leg|a|n|vn|ppp|ij|N|fp|I/P|vt|Im|Gu|dg|vi|io|px|sx|Ka|Kk|Ps|Kv|IG|vi2|prg|pag|Ju|sst|nst|pa|pn|cj|Ti|vo|lst|sst|nst|Le|plu_O|plu|v_stat|v_stat2|v_sup|v_imp|imp|Le|Mo|Ki|Mt|Aj|adv'.split('|')

In [2]:
"""Запускать вот это"""
deleteex = []
with open('newbeautiful14.txt', encoding='utf-8') as f:
    text = f.read()

# Этот кусок кода вычищает примеры употребления (: ~) и ссылки на другие лексемы (see)
# Вычищается не все-все, а только последовательность до следующей цифры, т.к. в большинстве случаев
# с цифры начинается новое значение, которое нужно оставить
for t in text.splitlines():
    t1 = re.sub('(:.+? )(\d\d?)', r' \2', t)
    t2 = re.sub('([sS]ee .+?)(\d\d?)', r'\2', t1)
    t21 = re.sub(' [23456789;†]$', '', t2)
    t3 = re.sub(';([^;\d]+)?~([^;$\d]+)?', ' ', t21)
    t4 = re.sub('\d [^\d]+~[^\d]+', '', t3)
    t5 = re.sub(':.+$', '', t4)
    t51 = re.sub(' [23456789;†]$', '', t5)
    deleteex.append(re.sub('[sS]ee .+?$', '', t51))

# Перезаписываю файл
with open('deleted2.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(deleteex))
    
with open('deleted2.txt', encoding='utf-8') as f:
    text = f.read().splitlines()

# вычищаю дублирующиеся скобки, которые помешают разбору
# также убираю падежи
newlines = []
for line in text:  
    line = re.sub('([A-z<])\(([A-z]+)\)([A-z-])', r'\1\2\3', line)
    line = re.sub('\(<[^\(\)]+\)', '', line) #(erg ramaC, dat rasaC, gen risaC, inst riTaC, adv radaC)
    line = re.sub('\((erg|dat|gen|inst|adv)[^\(\)]+\)', '', line)
    newlines.append(line)

# Перезаписываю вычищенный файл
with open('deleted3.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(newlines))
    
# Работаю с вычищенным   
with open('deleted3.txt', encoding='utf-8') as f:
    text = f.read().splitlines()    
   
another_turn = []
first_dict = {}
for line in text:
    pr = 0
    test = re.search('^([A-z]+) ([A-z]+)$', line)
    if test:
        if test.group(2) not in tags:
            first_dict[test.group(1)] = {'transl':test.group(2), 'lex_feat':None}
            print(line)
            pr = 1
    if pr == 0:
        another_turn.append(line)
        
another_turn2 = []
for line in another_turn:
    pr = 0
    test = re.search('^([A-z]+) ([A-z]+) ([A-z]+)$', line)
    if test:
        if test.group(2) in tags:
            if test.group(3) not in tags:
                #print(line)
                first_dict[test.group(1)] = {'transl':test.group(3), 'lex_feat':test.group(2)}
                #first_dict[test.group(1)] = test.group(2)
                pr = 1
            else:
                pass
    if pr == 0:
        another_turn2.append(line)
        
another_turn3 = []
for line in another_turn2:
    items = line.split(' ')
    test = all(m in tags for m in items[1:])
    if test:
        pass
    else:
        another_turn3.append(line)

verbsadjothers = []
turn4 = []
vns = []  
flexions = [] #аффиксы, которые пока отбрасываются из разбора

for line in another_turn3:
    first_tok = line.split(' ')[0]
    if re.search('[-~],?$', first_tok) or re.search('^[-~]', first_tok) or re.search('[A-z]~~[A-z]', first_tok):
        flexions.append(line)
    elif re.search(' (n|N|adv|ppp|pp|pa|ij|cj|pn|np|fp|imp)[ /]', line):
        turn4.append(line)
    elif re.search(' vn ', line):
        vns.append(line)
    else:
        verbsadjothers.append(line)

aberaCia aberration
absolutizmi absolutism
albomin album
alebastri alabaster
alegoria allegory
alegoriuli allegorical
alKimikosin alchemist
arKidiakoni archdeacon
arKiepiskoposi archbishop
aGxurviloba equipment


In [3]:
def find_alts(string):
    """Ищет ссылки на лексемы типа (<amo(s)jvreba) """
    alts = re.search('\(<([A-z ,-]+?)\)', string)
    if alts:
        alts = alts.group(1).split(',')
        alts = [alt.replace(' ', '') for alt in alts]
        alts = [re.sub(',.+', '', alt) for alt in alts]
    return alts

In [4]:
vnsdict = {}
difficult = []
for vn in vns:
    try:
       
        lexs, defs = vn.split(' vn ') 
        check = re.search('\(<([A-z ,\(-]+?)\)', defs)
        all_lexs = []
        
        
     # чищу ссылки на лексемы типа (<amo(s)jvreba) от скобок
       
        if check: # if there are differenet forms in definitions
            
            change = check.group(1).replace('(', '').replace(')', '')  #fix () inside definitions
            if change!= check.group(1):
                defs = defs.replace(check.group(1), change)
                defs = re.sub('([A-z])\)([A-z])', r'\1\2', defs)
        
        # если несколько значений, буду разбирать отдельно
        if re.search(' \d ?', lexs):
            difficult.append(vn)  
        else:
            
            changed_lexs = re.sub(' ?\(<([A-z][A-z][A-z][A-z].+)?\)', r', \1', lexs) # удаляю ссылки на лексемы типа (<amo(s)jvreba)
            dif = changed_lexs.split(',')
            dif = [d.replace('(', '').replace(')', '') for d in dif]
            new_dif = []
            # ищу теги в последовательностях
            for d in dif:
                dif_tags = []
                word_parts = []
                parts = d.strip(' ').split(' ')
                stop = None
                for i in range(len(parts)):
                    if parts[-i-1] in tags and stop == None:
                        dif_tags.append(parts[-i-1])
                    elif stop == None:
                        stop = -i
                if stop == 0:
                    word = d.strip(' ').replace(' ', '')
                else:
                    word = ''.join(parts[:stop])
                new_dif.append([word, dif_tags])
           
              
            alts = find_alts(defs)
            definitions = re.split('\d\d?', defs)
            # вычищаю определения от ссылок на другие лексемы
            definitions = [re.sub('\(<([A-z ,-]+?)\)', '', defi) for defi  in definitions if len(re.sub('\(<([A-z ,-]+?)\)', '', defi)) > 3]
           
                
            for d in new_dif:
                translation = ' #def '.join(definitions).strip(' ')
                
                dif_tags = []
                def_parts = []
                parts = translation.split(' ')
                stop = None
                for i in range(len(parts)):
                    if parts[i] in tags and stop == None:
                        dif_tags.append(parts[i])
                    elif stop == None:
                        stop = i
                tag_string = ' '.join(dif_tags)
                translation = re.sub(f'^{tag_string}', '', translation)
                
                if len(d[1]) < 1:
                    feats = 'vn' + ' ' + tag_string
                else:
                    feats = 'vn' + ' ' + ' '.join(d[1])
                vnsdict[d[0]] = {'transl':translation.strip(' '), 'lex_feat':feats}
            
        
    except ValueError:
        difficult.append(vn)

In [5]:
def find_alts2(string, position):
    """получение альтернативных форм глагола из скобок"""
    if position == 0:
        check = re.search('\(([^\(\)]+), ([^\(\)]+)\)', string)
    else:
        check = re.search('^\(([^\(\)]+), ([^\(\)]+)\)', string)
    alts = []
    if check:
        first = check.group(1).strip(' ')
        second = check.group(2).strip(' ')
        if re.search(' ?no ', first):
            first = None
        elif re.search(' ?no ', second):
            second = None
        if first:
            alts.append(first)
        if second:
            alts.append(second)
    if alts:
        return alts
    return None

Разбираю сложные случаи с vn

In [6]:
for line in difficult:
    
    # чищу от лишнего (упоминания какого лица глагол ит.д)
    line = re.sub('([A-z])\(([A-z]+)\)([A-z])', r'\1\2\3', line)
    line = re.sub('\(<[A-z --,\.]+\) ', ' ', line)
    line = re.sub(' \([1|2]p .+?\)(,)', r'\1', line)

    parts = re.split(' \d\d? ', line)
  #  print(parts)
    alternatives = []
    new_parts = []
    for n, part in enumerate(parts):
        new_part = part.strip(' ')
        toks = part.split(' ')
        stop = None
        found_tags = []
        for tok in toks:
            if tok in tags and stop == None:
                new_part = re.sub(f'^{tok} ?', '', new_part)
                found_tags.append(tok)
            elif stop == None:
                stop = 1
        #alts = find_alts(new_part)
        #new_part = re.sub(' ?\(<([A-z][A-z][A-z][A-z].+)?\)', r', \1', new_part)
        
        alts2 = find_alts2(new_part, n)
        if alts2:
           # print(alts2)
            for alt in alts2:
                alternatives.append(alt)
                if n == 0:
                    new_part = re.sub(' ?\(([^\(\)]+),([^\(\)]+)\)', '', new_part)
                else:
                    new_part = re.sub('^ ?\(([^\(\)]+),([^\(\)]+)\)', '', new_part)
        part = ' '.join(found_tags) + ' ' + new_part
        new_parts.append(part.strip(' '))
 #   print('*'*10,new_parts, alternatives)
    lex_parts = new_parts[0].split(' ')
    
    dif_tags = []

    stop = None
    for i in range(len(lex_parts)):
        if lex_parts[-i-1] in tags and stop == None:
            dif_tags.append(lex_parts[-i-1])
        elif stop == None:
            stop = -i
    tag_string = ' '.join(dif_tags)
    
    for tag in dif_tags:
        new_parts[0] = re.sub(f' ?{tag} ?', '', new_parts[0])
    
    definitions = [re.sub('^ ?vn ?', '', p) for p in new_parts[1:]]
    translation = ' #def '.join(definitions).strip(' ')
                
    

    feats = 'vn' + ' ' + tag_string
   
    vnsdict[new_parts[0]] = {'transl':translation.strip(' '), 'lex_feat':feats}
    if alts2:        
        for alt in alts2:
                vnsdict[alt] = {'transl':translation.strip(' '), 'lex_feat':feats}
    

c:\users\mjo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Possible set difference at position 8
  """


In [7]:
len(vnsdict) #15657

15657

In [8]:
#import json
#fname = 'parsedvns.json'
#with open(fname, 'w', encoding='utf-8') as f:
#    json.dump(vnsdict, f, indent = 6, ensure_ascii=False)

##### turn4

! Возможно, что для скобки в строке нужно заменять не так string = string.strip(' ').replace('(', '').replace(')', '')

А так:
```
string = string.strip(' ')
string = re.sub('([A-z])\) ', r'\1 ', string)
string = re.sub(' \(([A-z])', r' \1', string)
```

Но я пока не решила, т.к. есть слова типа ar(a), и там скобку нужно оставить, в отличие от, например, (sst)

In [9]:
def find_tags(string):
    """Выделение из подстроки, определенной как лексема, тегов """
    string = string.strip(' ').replace('(', '').replace(')', '')
    toks = string.split(' ')
    stop = None
    found_tags = []
    for i in range(len(toks)-1):
        if toks[-i-1] in tags and stop == None:
            found_tags.append(toks[-i-1])
        elif stop == None:
            stop = -i
    if stop == 0:
        lex = ''.join(toks)
    elif stop == None:
        lex = toks[0]
    else:
        lex = ''.join(toks[:stop])
    
    # чищу лексему в словаре от лишних символов (все, кроме букв, *, [], () удаляется)
    lex = re.sub('[^A-z-*\[\]\(\)]', '', lex)
    return lex, found_tags[::-1]       

In [10]:
def with_other_definitions(lex, other_defs):
    variants = lex.split(',')
    found_vars_tags = []
    for var in variants:
        var = var.replace('!', ',')
        
        v, t = find_tags(var)
        found_vars_tags.append([v, t])
        
    for v, t in found_vars_tags:
        if t:
            lex_f = ' '.join(t)
        else:
            lex_f = None
        translation = ' #def '.join(other_defs).strip(' ')
        
        first_dict[v] = {'transl':translation, 'lex_feat':lex_f}

In [11]:
def split_line(string):
    # если попался аффикс, напечатаем его. Большинство мы отсеяли в переменную flexions
    if re.search('[^A-z][ps]x[^A-z]', string):
        print('AFFIXSTRING:', string)
    toks = string.split(' ')
    start = None
    stop = None
    found_tags = []
    lexemes = []
    for i in range(len(toks)):
        if toks[i] not in tags and stop == None and start == None:
            lexemes.append(toks[i])
        elif toks[i] in tags and stop == None and start == None:
            start = i
            found_tags.append(toks[i])
        elif toks[i] in tags and stop == None and start != None:
            found_tags.append(toks[i])
        elif toks[i] not in tags and stop == None and start != None:
            stop = i
    lexs = ''.join(lexemes)
    definitions = ' '.join(toks[stop:])
    tag_string = ' '.join(found_tags)

    bracketes = re.findall('\(([^\(\)]+)\)', lexs)
    for b in bracketes:
        if b not in tags:
            lexs = re.sub(f'\(({b})\)', r'\1', lexs)
    
    lexs = lexs.replace('(', ' (').replace('!', ',')
 
    variants = lexs.split(',')
    forreturn = ''
    found_vars_tags = []
    for var in variants:
        v, t = find_tags(var)
        found_vars_tags.append([v, t])
        for v, t in found_vars_tags:

            if t:
                lex_f = tag_string + ' ' + ' '.join(t)
            else:
                lex_f = tag_string
            translation = definitions
            
            # чистка от вхождения типа 'lexeme ... 1 definition 2', т.е. когда обрывается определение
            if not re.sub('2+$', '', translation):
                first_dict[v] = {'transl':translation, 'lex_feat':lex_f}
                forreturn += f'|||{v}|||{translation}|||{lex_f}\n'
            else:
                first_dict[v] = {'transl':re.sub('2+$', '', translation), 'lex_feat':lex_f}
                forreturn += f'|||{v}|||{translation}|||{lex_f}\n'
            
            if re.search('[^A-z][ps]x[^A-z]', string):
                print(v, '|transl|', translation, '|lex_f|', lex_f)
    return forreturn

In [12]:
difficultdigits = [] # выбрасываю строки с ошибками в порядке значений 1 2 2 3 или строки с цифровыми значениями (88 литров)
turn5 = []
for turn in turn4:
    digits = re.findall(' \d\d? ', turn)
    summa = 0
    for i in range(len(digits)):
        summa += i+1
    if summa != sum([int(d) for d in digits]):
        difficultdigits.append(turn)
    else:
        turn5.append(turn)

In [13]:
for turn in turn5:
    parts = re.split(' \d\d? ', turn)
    lex = parts[0]
    other_defs = parts[1:]
    
    if other_defs:
        with_other_definitions(lex, other_defs)
       
            
    else: ## если только одно значение в строке
        split_line(lex)

AFFIXSTRING: arKe(~) n px ‡ chief, leader, arch~
arKe |transl| chief, leader, arch~ |lex_f| n px ‡
AFFIXSTRING: xkva(~) n px nst wits, mind 
xkva |transl| wits, mind  |lex_f| n px nst
AFFIXSTRING: Hanmeti n a Khanmet’i (middle stage of Old Georgian c 600 AD, when H- px marked verb’s 2p S & 3p IO) 
Hanmeti |transl| Khanmet’i (middle stage of Old Georgian c 600 AD, when H- px marked verb’s 2p S & 3p IO)  |lex_f| n a
AFFIXSTRING: haemeti n (in) early Old Georgian (haemet’i, extra ‘h’, h- as px marking 2p S and 3p IO of all verbs) 
haemeti |transl| (in) early Old Georgian (haemet’i, extra ‘h’, h- as px marking 2p S and 3p IO of all verbs)  |lex_f| n


In [14]:
other = []
verbs = []
iofp = []

for item in verbsadjothers:
    if re.search(' (io|fp) ', item) and not re.search(' v[istco_]', item):
        iofp.append(item)
    elif not re.search(' v[istco_][^A-z]', item) and not re.search(' aor ', item) and not re.search(' will ', item):
        other.append(item)
    else:
        verbs.append(item)

In [15]:
for turn in iofp:
    parts = re.split(' \d\d? ', turn)
    lex = parts[0]
    other_defs = parts[1:]
    
    if other_defs:
        with_other_definitions(lex, other_defs)
    else: 
        split_line(lex)   

##### other

In [16]:
verbforms = []
other2 = []
adjdifficultdigits = []
adjlen1 = []
notags = []  # нет тегов в строке
gooddigits = []
eba = []  # скорее всего строки vn, в которых по каким-то причинам отсутствует тег vn
for o in other:
    if re.search('^[A-z\[\]\(\)]+(s|vs|bian|eva)[ ,]', o):
       verbforms.append(o)
    elif re.search('^[A-z\[\]\(\)]+eba[ ,]', o):
        if 'a' in o.split(' ')[:6]:  # если в начале встречается тег прилагательного, не буду кидать это слово к герундиям-vn
            other2.append(o)
        else:
            eba.append(o)    
    elif re.search('[\(\)A-z-]+ +\([\(\)A-z-]+, [\(\)A-z-]+\)', o.replace('1', ' ').replace('(†)', ' ')):
        first_occ = re.search('([\(\)A-z-]+) +\([\(\)A-z-]+, [\(\)A-z-]+\)', o.replace('1', ' ').replace('(†)', ' ')).group(1)
        if len(first_occ) < 2 and first_occ != 'a':
            verbforms.append(o)
        elif re.search('[A-z\[\]\(\)]+(s|ba|vs|bian)$', first_occ):
            verbforms.append(o)
        else:
            other2.append(o)
    
    elif re.search('(fut|pres|aor|imp|2p|IO|3p|pf)[^A-z]', o) and not re.search(' [p|s]x ', o):
        verbforms.append(o)
    else:
       # print(o)
        found_tags = []
        toks = o.split(' ')[:7]
        for tok in toks:
            if tok.replace('(', '').replace(')', '') in tags:
                found_tags.append(tok)
        digits = re.findall('\d\d?', o)
        summa = 0
        for i in range(len(digits)):
            summa += i+1
        if summa == sum([int(d) for d in digits]) and summa > 0:
            gooddigits.append(o)
        elif summa != sum([int(d) for d in digits]):
            if len(digits)==1:
                adjlen1.append(o)
            else:
                adjdifficultdigits.append(o)
        
        
        elif len(found_tags) == 0:
            notags.append(o)
        else:
            other2.append(o)
    

Разберусь с presumbaly герундиями

In [17]:
for line in eba:
    
    if re.search('^([A-z-]+) \(([A-z-]+), ([A-z -]+)\)', line):
        query = re.search('^([A-z-]+) \(([A-z-]+), ([A-z -]+)\)', line)
        lexemes = [query.group(1), query.group(2)]
        if query.group(3) != 'no pf':
            lexemes.append(query.group(3))
        definitions = line.replace(query.group(0), '')
        
        if len(re.split(' \d\d? ', definitions)) >1:
            definitions = re.split(' \d\d? ', definitions)
        else:
            defs = []
            defs.append(definitions)
            definitions = defs
            
        f_tags = []
        stop = None
        tokens = definitions[0].strip(' ').split(' ')
        for n, token in enumerate(tokens):
            if token in tags and stop == None:
                f_tags.append(token)
            elif stop == None:
                stop = n
        definitions[0] = ' '.join(tokens[stop:])
       # print(definitions)
        translation = ' #def '.join(definitions)
        for lexeme in lexemes:
            if translation:
                first_dict[lexeme] = {'transl': translation, 'lex_feat': ' '.join(tags)}
    else:
        
        if len(re.split(' \d\d? ', line)) == 1:
            other2.append(line)
        else:
            parts = re.split(' \d\d? ', line)

            alternatives = []
            new_parts = []
            for n, part in enumerate(parts):
                new_part = part.strip(' ')
                toks = part.split(' ')
                stop = None
                found_tags = []
                for tok in toks:
                    if tok in tags and stop == None:
                        new_part = re.sub(f'^{tok} ?', '', new_part)
                        found_tags.append(tok)
                    elif stop == None:
                        stop = 1
                
                
                alts2 = find_alts2(new_part, n)
                if alts2:
                    for alt in alts2:
                        alternatives.append(alt)
                        if n == 0:
                            new_part = re.sub(' ?\(([^\(\)]+),([^\(\)]+)\)', '', new_part)
                        else:
                            new_part = re.sub('^ ?\(([^\(\)]+),([^\(\)]+)\)', '', new_part)
                part = ' '.join(found_tags) + ' ' + new_part
                new_parts.append(part.strip(' '))
            lex_parts = new_parts[0].split(' ')
            
            dif_tags = []

            stop = None
            for i in range(len(lex_parts)):
                if lex_parts[-i-1] in tags and stop == None:
                    dif_tags.append(lex_parts[-i-1])
                elif stop == None:
                    stop = -i
            tag_string = ' '.join(dif_tags)
            
            for tag in dif_tags:
                new_parts[0] = re.sub(f' ?{tag} ?', '', new_parts[0])
            
            
            translation = ' #def '.join([defi for defi in definitions if len(defi)>0]).strip(' ')
                        
            
    
            feats = tag_string
            if translation:
                first_dict[new_parts[0]] = {'transl':translation.strip(' '), 'lex_feat':feats}
               # print(new_parts[0], translation)

In [18]:
for turn in other2:
    parts = re.split(' \d\d? ', turn)
    lex = parts[0]
    other_defs = parts[1:]
    
    if other_defs:
        with_other_definitions(lex, other_defs)
    else:
        split_line(lex)

AFFIXSTRING: (-)dagvarad sx hyphenated after gen of noun similarly; accordingly
-dagvarad |transl| hyphenated after gen of noun similarly; accordingly |lex_f| sx


In [19]:
for turn in adjlen1:
    lex = turn
    other_defs = None
    if other_defs:
        with_other_definitions(lex, other_defs)
    else:
        split_line(lex)

In [20]:
for turn in gooddigits:
    parts = re.split(' \d\d? ', turn)
    lex = parts[0]
    other_defs = parts[1:]
    
    if other_defs:
        with_other_definitions(lex, other_defs)
    else:
        split_line(lex)

In [21]:
len(first_dict) #83968

85276

In [22]:
len(verbforms) #2865

3341

In [23]:
len(adjdifficultdigits) #71

75

In [24]:
len(difficultdigits) #491

491

In [25]:
for turn in difficultdigits:
    
    digits = re.findall(' \d\d? ', turn)
    summa = 0
    digits = [int(d) for d in digits]
    dif = []
    accepted = [0, -1, -2]
    
    if len(digits)>1:
        
        for i in range(len(digits)-1):
           
            dif.append(digits[i] - digits[i+1])
            
    check = all([d in accepted for d in set(dif)]) and dif
    
    if len(digits) == 1 and digits[0] == 2 and not re.search('kopecs', turn):
        
        new_t = turn.replace(' 2 ', ' #def ')
        new_t = re.sub('#def $', '', new_t)
        split_line(new_t)
        
    elif check:
        parts = re.split(' \d ', turn)
        lex = parts[0].strip(' ')
        other_defs = parts[1:]
        
        
        lex_toks = lex.split(' ')
        if len(lex_toks) >1:
            
            lexeme_end = 0
            tag_start = 0
            ended = 0
            
            for n, tok in enumerate(lex_toks):
                

                
                if tok in tags and tag_start == 0:
                    #print(tok, n)
                    lexeme_end = n
                    tag_start = 1
                elif tok in tags and ended == 0:
                    lexeme_end = n
                elif tok not in tags and tag_start == 1 and ended == 0:
                    lexeme_end = n
                    ended = 1
    
 
            lex = ' '.join(lex_toks[:lexeme_end+1])
        
          
            try:
                parts[1] = ' '.join(lex_toks[lexeme_end+1:]) + ' ' + parts[1]
            except IndexError:
                parts.append(' '.join(lex_toks[lexeme_end+1:]) + ' ')

        
        
        if other_defs:
           # print('checked2:', turn)
            variants = lex.split(',')
            found_vars_tags = []
            for var in variants:
                var = var.replace('!', ',')
                v, t = find_tags(var)
                found_vars_tags.append([v, t])
            for v, t in found_vars_tags:
                if t:
                    lex_f = ' '.join(t)
                else:
                    lex_f = None
                translation = ' #def '.join(other_defs).strip(' ')
                #print(turn, '|||', v, '|||', translation, '|||', lex_f)
                first_dict[v] = {'transl':translation, 'lex_feat':lex_f}
        

    elif dif:
        parts = re.split('[ ;][12345789] ', turn)
        if not len(parts)>1:
            split_line(turn)
        else:
            lex = parts[0]
            other_defs = parts[1:]
            if other_defs:
                variants = lex.split(',')
                found_vars_tags = []
                for var in variants:
                    var = var.replace('!', ',')
                    v, t = find_tags(var)
                    found_vars_tags.append([v, t])
                for v, t in found_vars_tags:
                    if t:
                        lex_f = ' '.join(t)
                    else:
                        lex_f = None
                    translation = ' #def '.join(other_defs).strip(' ')
                    
                    first_dict[v] = {'transl':translation, 'lex_feat':lex_f}
    
    #print(turn)
    else:
        split_line(turn)

In [26]:
len(first_dict) + len(vnsdict) #100130

101438

In [27]:
c = 0
kv = 0
for turn in adjdifficultdigits:
    
    digits = re.findall(' \d\d? ', turn)
    summa = 0
    digits = [int(d) for d in digits]
    dif = []
    accepted = [0, -1, -2]
    
    if len(digits)>1:
        for i in range(len(digits)-1):
            dif.append(digits[i] - digits[i+1])
            
    check = all([d in accepted for d in set(dif)]) and dif
    
    if len(digits) == 1 and digits[0] == 2 and not re.search('kopecs', turn):
        new_t = turn.replace(' 2 ', ' #def ')
        split_line(new_t)
        print(split_line(new_t))
        
        
 ##############################################################       
    elif check:
        parts = re.split(' \d ', turn)
        lex = parts[0].strip(' ')
     #   print(lex)
        other_defs = parts[1:]
        
        
        lex_toks = lex.split(' ')
        if len(lex_toks) >1:
            
            lexeme_end = 0
            tag_start = 0
            ended = 0
            for n, tok in enumerate(lex_toks):

                if tok in tags and tag_start == 0:
                    #print(tok, n)
                    lexeme_end = n
                    tag_start = 1
                elif tok in tags and ended == 0:
                    lexeme_end = n
                elif tok not in tags and tag_start == 1 and ended == 0:
                    lexeme_end = n
                    ended = 1
          #  print(lexeme_end)   
            lex = ' '.join(lex_toks[:lexeme_end+1])
        
        
 ###################################################################       
            try:
                parts[1] = ' '.join(lex_toks[lexeme_end+1:]) + ' ' + parts[1]
            except IndexError:
                parts.append(' '.join(lex_toks[lexeme_end+1:]) + ' ')

        if other_defs:
            variants = lex.split(',')
            found_vars_tags = []
            for var in variants:
                var = var.replace('!', ',')
                v, t = find_tags(var)
                found_vars_tags.append([v, t])
            for v, t in found_vars_tags:
                if t:
                    lex_f = ' '.join(t)
                else:
                    lex_f = None
                translation = ' #def '.join(other_defs).strip(' ')
                #print(turn, '|||', v, '|||', translation, '|||', lex_f)
                first_dict[v] = {'transl':translation, 'lex_feat':lex_f}
               # print(v, first_dict[v])
   # elif all([d for d in set(dif) in accepted]):
   #     print(t)
    elif dif:
        parts = re.split('[ ;][12345789] ', turn)
        if not len(parts)>1:
            split_line(turn)
            print(split_line(turn))
        else:
            lex = parts[0]
            other_defs = parts[1:]
            if other_defs:
                variants = lex.split(',')
                found_vars_tags = []
                for var in variants:
                    var = var.replace('!', ',')
                    v, t = find_tags(var)
                    found_vars_tags.append([v, t])
                for v, t in found_vars_tags:
                    if t:
                        lex_f = ' '.join(t)
                    else:
                        lex_f = None
                    translation = ' #def '.join(other_defs).strip(' ')
                    
                    first_dict[v] = {'transl':translation, 'lex_feat':lex_f}
                   
    else:
        split_line(turn)

In [28]:
for line in notags:
    tokens = line.strip(' ').split(' ')
    lex_tokens = []
    def_tokens = []
    border = None
    bordered = False
    for n, token in enumerate(tokens):
        if not re.search('[,!]$', token) and bordered == False:
            border = n
            bordered = True
    lexemes = tokens[:border+1]
    defs = tokens[border+1:]
    translation = ' '.join([defi for defi in defs if len(defi)>0])
    for lexeme in lexemes:
        first_dict[lexeme] = {'transl':translation, 'lex_feat':""}

Не забыть добавить эту строку, не помню по какой причине, но это в словарь не попадало

In [46]:
first_dict['eli'] = {'transl':'n bot © azalea #def N Turkic nomadic tribe; #def n † land, country; #def † herd of cattle',
                     'lex_feat':""}
first_dict.update(vnsdict)

In [61]:
first_filtered = {}

for key in first_dict.keys():
    if not first_dict[key]['transl'] == '':
        if first_dict[key]['transl'] == None:
            first_dict[key]['transl'] = ""
        elif first_dict[key]['lex_feat'] == None:
            first_dict[key]['lex_feat'] = ""
        first_dict[key]['transl'] = first_dict[key]['transl'].strip(' ')
        first_dict[key]['lex_feat'] = first_dict[key]['lex_feat'].strip(' ')
        first_filtered[key] = first_dict[key]
first_dict = first_filtered

In [62]:
len(first_dict)

101550

In [56]:
import json
fname = 'parsed11.json'
with open(fname, 'w', encoding='utf-8') as f:
    json.dump(first_dict, f, indent = 6, ensure_ascii=False)

In [35]:
for turn in turn5:
    parts = re.split(' \d\d? ', turn)
    lex = parts[0]
    other_defs = parts[1:]
    
    if other_defs:
        with_other_definitions(lex, other_defs)
       
            
    else: ## если только одно значение в строке
        split_line(lex)

AFFIXSTRING: arKe(~) n px ‡ chief, leader, arch~
arKe |transl| chief, leader, arch~ |lex_f| n px ‡
AFFIXSTRING: xkva(~) n px nst wits, mind 
xkva |transl| wits, mind  |lex_f| n px nst
AFFIXSTRING: Hanmeti n a Khanmet’i (middle stage of Old Georgian c 600 AD, when H- px marked verb’s 2p S & 3p IO) 
Hanmeti |transl| Khanmet’i (middle stage of Old Georgian c 600 AD, when H- px marked verb’s 2p S & 3p IO)  |lex_f| n a
AFFIXSTRING: haemeti n (in) early Old Georgian (haemet’i, extra ‘h’, h- as px marking 2p S and 3p IO of all verbs) 
haemeti |transl| (in) early Old Georgian (haemet’i, extra ‘h’, h- as px marking 2p S and 3p IO of all verbs)  |lex_f| n


Ура, молодец, распарсила почти весь словарь! Не у дел только:

verbforms

verbs

flexions

#### convert georgian

In [58]:
alp ='''aა
bბ
gგ
dდ
eე
vვ
zზ
**Eჱ
Tთ
iი
kკ
lლ
mმ
nნ
**yჲ
oო
pპ
Zჟ
rრ
sს
tტ
uუ
**wჳ
Pფ
Kქ
Gღ
qყ
Sშ
Xჩ
Cც
jძ
cწ
xჭ
Hხ
**Qჴ
Jჯ
hჰ
**Oჵ'''
alpdict = {}
for string in alp.splitlines():
    trans, georg = list(string.strip('*'))
    alpdict[trans] = georg

Конвертирую словарь по правилам национального корпуса

In [63]:
georg_dict = {}
for key in first_dict.keys():
    transl = ''
    for i in list(key):
        try:
            t = alpdict[i]
        except KeyError:
            t = i
        transl += t
    tran = first_dict[key]['transl']
    tr1 = re.sub('^ ?#def ?', '', tran)
    tr2 = re.sub(' ?#def ?$', '', tr1)
    first_dict[key]['transl'] = tr2
    georg_dict[transl] = first_dict[key]
            

In [64]:
fname = 'parsed12.json'
with open(fname, 'w', encoding='utf-8') as f:
    json.dump(georg_dict, f, indent = 6, ensure_ascii=False)

Отдельно словарь без пунктуации, т.к. где-то расхождения между парсером Мойера и словарем, и преугадать не всегда получится.

In [66]:
no_punct_dicti = {}
for key in georg_dict.keys():
    newkey = ''
    for l in list(key):
        if l.isalpha():
            newkey += l
    no_punct_dicti[newkey] = georg_dict[key]
with open('parsed12_nopunct.json', 'w', encoding='utf-8') as f:
    json.dump(no_punct_dicti, f, indent = 6, ensure_ascii=False)